<a href="https://colab.research.google.com/github/aastle/Scripts/blob/main/examples/text2img.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Civitai SDK Text2Img Example
This notebook demonstrates how to generate an image using the Civitai SDK.

In [1]:
%pip install civitai-py
%pip install ipython

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.7/61.7 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 11.7 MB/s eta 0:00:00


In [ ]:
import os

api_token = input("api_key_goes_here")
os.environ['CIVITAI_API_TOKEN'] = api_token

### Example 1: Manually polling for status of job

In [18]:
# Import the Civitai SDK
import civitai

model = "urn:air:sdxl:checkpoint:civitai:312530@2391289"
positivePrompt = ("3D, face portrait, 3D, color image of a woman with black, curly hair, wearing teal and red polkadot spots jacket,"
                  " worried face, standing in front of a log cabin, soft shadows, cinematic shot, diffuse lighting")
#EulerA, Euler, LMS, Heun, DPM2, DPM2A, DPM2SA, DPM2M, DPMSDE, DPMFast, DPMAdaptive, LMSKarras, DPM2Karras,
#DPM2AKarras, DPM2SAKarras, DPM2MKarras, DPMSDEKarras, DDIM, PLMS, UniPC, Undefined, LCM, DDPM, DEIS.
scheduler = "DPM2SAKarras"
negativePrompt = "(deformed, distorted, disfigured:1.3)"
steps = 26
cfgscale = 6
width = 768
height = 512
seed = 1847995
clipSkip = 2

#Extra Networkds Parameters assigned here:
loraAir = "urn:air:sdxl:lora:civitai:73756@703107"
xtraNetworkType = "Lora"
xtraNetworkStrength = 1.0
xtraNetworkTriggerWord = "3DMM"

# Define the input parameters for the image creation
input = {
    "model": model,
    "params": {
        "prompt": positivePrompt,
        "negativePrompt": negativePrompt,
        "scheduler": scheduler,
        "steps": steps,
        "cfgScale": cfgscale,
        "width": width,
        "height": height,
        "seed": seed,
        "clipSkip": clipSkip
    },
    "additionalNetworks": {
        loraAir: {
            "type": xtraNetworkType,
            "strength": xtraNetworkStrength,
            "triggerWord" : xtraNetworkTriggerWord
        }
    }
}

# Generate the image
# We're using await in the notebook to handle the coroutine
response = await civitai.image.create(input)
print("Response:", response)

Response: {'token': 'eyJKb2JzIjpbIjMzNzI0YWY5LWEzNWItNGY0NS1iNjY1LWZjOGIxNjA4MjRmNCJdfQ==', 'jobs': [{'jobId': '33724af9-a35b-4f45-b665-fc8b160824f4', 'cost': 1.8719999999999999, 'result': [{'blobKey': '0F60DE1C7E31F1CDC9930193BA1785C46513478F0A0564060F944328FF928839', 'available': False, 'seed': 1847995}], 'scheduled': True}]}


You can then retrieve the job status and image using the job token or job ID.

In [26]:
from IPython.display import Image, display

job_token = response['token']
job_id = response['jobs'][0]['jobId']

# Retrieve job status and image
# We're using await in the notebook to handle the coroutine
response = await civitai.jobs.get(token=job_token, job_id=job_id)
print("Job Status Response:", response)

jsonResponse = response['jobs'][0]['result']
print("json part of the response:", jsonResponse)

imageAvailable = jsonResponse[0].get('available')
print("Is image available:", imageAvailable)

if imageAvailable:
    image_url = jsonResponse[0].get('blobUrl')
    if image_url:
        display(Image(url=image_url))
    else:
        print("Image URL not found in the job result.")
    #print("Image URL availale, check the response text.")
else:
   print("No image was created, the job is not yet complete, or the result is not available.")

Job Status Response: {'token': 'eyJKb2JzIjpbIjMzNzI0YWY5LWEzNWItNGY0NS1iNjY1LWZjOGIxNjA4MjRmNCJdfQ==', 'jobs': [{'jobId': '33724af9-a35b-4f45-b665-fc8b160824f4', 'cost': 1.8719999999999999, 'result': [{'blobKey': '0F60DE1C7E31F1CDC9930193BA1785C46513478F0A0564060F944328FF928839', 'available': False, 'seed': 1847995}], 'scheduled': True}]}
json part of the response: [{'blobKey': '0F60DE1C7E31F1CDC9930193BA1785C46513478F0A0564060F944328FF928839', 'available': False, 'seed': 1847995}]
Is image available: False
No image was created, the job is not yet complete, or the result is not available.
